# Final Project — Prayer Rakats Finder (Simple)

**Goal:** Type a prayer name → see how many rakʿāt it has.

**You will:**
1) Run a tiny Flask API.  
2) Open an HTML page.  
3) Type the prayer name and click the button.


## 0) Install (if needed)

In [ ]:
# Run once if Flask is missing
try:
    import flask  # noqa: F401
    print("✅ Flask is already installed!")
except Exception:
    import sys, subprocess
    print("Installing Flask...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "flask"])
    print("✅ Flask installed!")

## 1) Make the API (skeleton)
This writes **`app.py`**. Run it with `python app.py`.


In [ ]:
%%writefile app.py
from flask import Flask, jsonify, request, make_response

app = Flask(__name__)

# 👉 Edit this if you want
PRAYER_RAKATS = {
    "fajr": 2,
    "dhuhr": 4,
    # add other prayers
}

def allow_cors(resp):
    resp.headers["Access-Control-Allow-Origin"] = "*"
    return resp

def normalize(name: str) -> str:
    if not name:
        return ""
    name = name.strip().lower()
    # Bonus: handle alias
    if name == "zuhr":
        name = "dhuhr"
    return name

@app.get("/api/rakats")
def rakats():
    raw = request.args.get("name", "")
    name = normalize(raw)
    if not name:
        return allow_cors(make_response(jsonify({"error": "Type a prayer name, e.g., Fajr"}), 400))
    r = PRAYER_RAKATS.get(name)
    if r is None:
        return allow_cors(make_response(jsonify({"error": "Unknown prayer. Try Fajr, Dhuhr/Zuhr, Asr, Maghrib, Isha"}), 404))
    return allow_cors(make_response(jsonify({"prayer": name, "rakats": r})))
    
@app.get("/")
def home():
    return "<h3>Try /api/rakats?name=fajr</h3>"

if __name__ == "__main__":
    app.run(debug=True)

## 2) Open the Page
This writes **`find_rakats.html`**. Open it in your browser.


In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>Prayer Rakats Finder</title>
  <style>
    body { font-family: Arial, sans-serif; padding: 20px; }
    input, button { padding: 10px; font-size: 16px; }
    #msg { margin-top: 10px; font-weight: bold; }
  </style>
</head>
<body>
  <h2>🕌 Prayer Rakats Finder</h2>
  <p>Type a prayer name (Fajr, Dhuhr, Asr, Maghrib, Isha)</p>
  <input id="inp" placeholder="e.g., Fajr"/>
  <button id="go">Find</button>
  <div id="msg"></div>

  <script>
    const API = "http://127.0.0.1:5000";
    const inp = document.getElementById('inp');
    const msg = document.getElementById('msg');

    function show(text, ok=true){
      msg.style.color = ok ? "green" : "red";
      msg.textContent = text;
    }

    async function findRakats(){
      const name = (inp.value || "").trim();
      if (!name){
        show("Please type a prayer name.", false);
        return;
      }
      show("Loading...");
      try{
        const r = await fetch(API + "/api/rakats?name=" + encodeURIComponent(name));
        const data = await r.json();
        if (!r.ok){
          show(data.error || "Error. Try again.", false);
          return;
        }
        show((data.prayer || name) + " has " + data.rakats + " rak\u02BF\u0101t.", true);
      }catch(e){
        show("Network error. Is the API running?", false);
        console.error(e);
      }
    }

    document.getElementById('go').addEventListener('click', findRakats);
    inp.addEventListener('keydown', e => { if (e.key === "Enter") findRakats(); });
  </script>
</body>
</html>

## 3) Your Turn
- Try different cases: `FAJR`, `fAjR` (API handles case).  
- Use alias: **Zuhr → Dhuhr** (already added).  
- Make the result text bigger or blue.  
- On PythonAnywhere, change `API` in HTML to your domain.
